<a href="https://colab.research.google.com/github/mohanrajmit/Bird-Classification-using-CNN/blob/master/bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization


Using TensorFlow backend.


In [2]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR10 has ten types of images labeled from 0 to 9. We only care about birds, which are labeled as class #2.
# So we'll cheat and change the Y values. Instead of each class being labeled from 0 to 9, we'll set it to True
# if it's a bird and False if it's not a bird.
y_train = (y_train == 2).astype(int)
y_test = (y_test == 2).astype(int)

170500096/170498071 [==============================] - 41s 0us/step


In [0]:
# Normalize image data (pixel values from 0 to 255) to the 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Create a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

In [6]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_model.h5")

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.1141 - acc: 0.9573 - val_loss: 0.2607 - val_acc: 0.9286
Epoch 2/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.1083 - acc: 0.9601 - val_loss: 0.1854 - val_acc: 0.9361
Epoch 3/10
50000/50000 [==============================] - 23s 461us/step - loss: 0.1018 - acc: 0.9621 - val_loss: 0.1836 - val_acc: 0.9379
Epoch 4/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.0952 - acc: 0.9646 - val_loss: 0.1948 - val_acc: 0.9415
Epoch 5/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.0926 - acc: 0.9662 - val_loss: 0.2609 - val_acc: 0.9334
Epoch 6/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.0899 - acc: 0.9674 - val_loss: 0.2147 - val_acc: 0.9380
Epoch 7/10
50000/50000 [==============================] - 23s 462us/step - loss: 0.0847 - acc: 0.9691 - val_loss: 0.

In [0]:
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import load_model

In [8]:
# Load the model we trained
model = load_model('bird_model.h5')
predictions = model.predict(x_test, batch_size=32, verbose=1)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

10000/10000 [==============================] - 1s 149us/step
True Positives: 495
True Negatives: 8914
False Positives: 86
False Negatives: 505
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      9000
           1       0.85      0.49      0.63      1000

   micro avg       0.94      0.94      0.94     10000
   macro avg       0.90      0.74      0.80     10000
weighted avg       0.94      0.94      0.93     10000

